In [1]:
years = """
<div class="filter__column column-1">
<select class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_season_id" id="filter_season_id">
<option value="0">Change Season</option><option value="24">2024</option>
<option value="23" selected="selected">2023</option>
<option value="22">2022</option>
<option value="21">2021</option>
<option value="10">2020</option>
<option value="9">2019</option>
<option value="8">2018</option>
<option value="7">2017</option>
<option value="6">2016</option>
<option value="5">2015</option>
<option value="1">2014</option>
<option value="2">2013</option>
<option value="3">2012</option>
<option value="4">2011</option>
<option value="11">2010</option>
<option value="12">2009</option>
<option value="13">2008</option>
<option value="14">2007</option>
<option value="15">2006</option>
<option value="16">2005</option>
<option value="17">2004</option>
<option value="18">2003</option>
<option value="19">2002</option>
<option value="20">2001</option>
</select>
</div>"""

race_traks = """
<div class="filter__column column-2">
<select id="filter_meeting_id" class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_meeting_id">
<option value="0">Meeting</option><option value="211" selected="selected">Monza</option>
<option value="197">Brands Hatch</option>
<option value="200">Circuit Paul Ricard 1000Km</option>
<option value="204">CrowdStrike 24 Hours of Spa</option>
<option value="203">Misano</option>
<option value="213">Nürburgring </option>
<option value="205">Hockenheim</option>
<option value="206">Valencia</option>
<option value="207">Barcelona</option>
<option value="201">Zandvoort</option>
</select>
</div>
"""

race = """
<div class="filter__column column-3">
<select id="filter_race_id" class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_race_id">
<option value="">Session</option>
<option value="1378">Race 2</option>
<option value="1377">Qualifying 2</option>
<option value="1376">Race 1</option>
<option value="1375">Qualifying 1</option>
<option value="1374">Pre Qualifying</option>
<option value="1373">Free practice</option>
<option value="1372">Official Paid Test Sessions 1</option>
</select> 
</div>
"""



In [11]:
import requests
from bs4 import BeautifulSoup

def fetch_initial_data(years=None):
    url = "https://www.gt-world-challenge-europe.com/results?filter_season_id=0&filter_meeting_id=0&filter_race_id="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Извлекаем доступные сезоны
    seasons_options = soup.select('#filter_season_id option')
    seasons = {option.text.strip(): option['value'] for option in seasons_options if option['value']}

    # Фильтруем сезоны по указанным годам, если таковые имеются
    if years:
        seasons = {year: seasons[year] for year in years if year in seasons}
    
    # Извлекаем доступные встречи
    meetings_options = soup.select('#filter_meeting_id option')
    meetings = {option.text.strip(): option['value'] for option in meetings_options if option['value']}
    
    return seasons, meetings


def parse_years(input_years):
    if isinstance(input_years, list):
        return [str(year) for year in input_years]
    elif isinstance(input_years, str) and '-' in input_years:
        start_year, end_year = map(int, input_years.split('-'))
        return [str(year) for year in range(start_year, end_year - 1, -1)]
    elif input_years:
        return [str(input_years)]
    else:
        return None


def get_race_data(input_years=None):
    base_url = "https://www.gt-world-challenge-europe.com/results"
    years = parse_years(input_years)
    seasons, meetings = fetch_initial_data(years)
    results = []

    for season_name, season_id in seasons.items():
        for meeting_name, meeting_id in meetings.items():
            url = f"{base_url}?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id="
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            race_options = soup.select('#filter_race_id option')

            races = []
            for option in race_options:
                if option['value']: # Исключаем пустые значения
                    races.append({"race_id": option['value'], "name": option.text.strip()})
            
            # Добавляем информацию о сезоне, встрече и гонках в результат
            results.append({
                "season": season_name,
                "season_id": season_id,
                "meeting": meeting_name,
                "meeting_id": meeting_id,
                "races": races
            })

    return results


In [12]:
# Примеры вызова функции:
#data_all_years = get_race_data()  # Данные за все годы
data_specific_year = get_race_data(2023)  # Данные только за 2023 год
#data_years_list = get_race_data([2023, 2022])  # Данные за 2023 и 2022 годы
#data_years_range = get_race_data("2023-2021")  # Данные за 2023, 2022 и 2021 годы

print(data_specific_year)

[{'season': '2023', 'season_id': '23', 'meeting': 'Meeting', 'meeting_id': '0', 'races': []}, {'season': '2023', 'season_id': '23', 'meeting': 'Monza', 'meeting_id': '211', 'races': [{'race_id': '1356', 'name': 'Main Race after 2.30 hours'}, {'race_id': '1355', 'name': 'Main Race after 1.30 hour'}, {'race_id': '1357', 'name': 'Main Race after 30 mins'}, {'race_id': '1329', 'name': 'Main Race'}, {'race_id': '1354', 'name': 'Combined Qualifying'}, {'race_id': '1353', 'name': 'Qualifying 3'}, {'race_id': '1352', 'name': 'Qualifying 2'}, {'race_id': '1328', 'name': 'Qualifying 1'}, {'race_id': '1351', 'name': 'Pit walk'}, {'race_id': '1350', 'name': 'Fanatec Esports GT Pro Series'}, {'race_id': '1327', 'name': 'Pre Qualifying'}, {'race_id': '1326', 'name': 'Free practice'}, {'race_id': '1325', 'name': 'Bronze Test'}]}, {'season': '2023', 'season_id': '23', 'meeting': 'Brands Hatch', 'meeting_id': '197', 'races': [{'race_id': '1324', 'name': 'Race 2'}, {'race_id': '1323', 'name': 'Race 1'},

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_race_data(filter_season_id, filter_meeting_id, filter_race_id):
    # Словари для хранения соответствий ID
    seasons = {
        2024: "24",
        2023: "23",
        2022: "22",
        # Добавьте остальные необходимые соответствия
    }
    meetings = {
        "Barcelona": "207",
        "Misano": "203",
        # Добавьте остальные необходимые соответствия
    }
    
    # Проверяем, есть ли указанные ID в словарях
    season_value = seasons.get(filter_season_id)
    meeting_value = meetings.get(filter_meeting_id)
    
    if season_value is None or meeting_value is None:
        raise ValueError("Указаны неверные значения для сезона или встречи.")
    
    # URL для первого запроса (для получения списка сессий)
    url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id='
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Поиск нужного filter_race_id по тексту ссылки
    race_options = soup.select('#filter_race_id option')
    race_value = None
    for option in race_options:
        if option.text.strip() == filter_race_id:
            race_value = option['value']
            break
    
    if race_value is None:
        raise ValueError(f"Сессия {filter_race_id} не найдена.")
    
    # URL для второго запроса (для получения данных сессии)
    final_url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id={race_value}'
    
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг данных сессии
    rows = soup.select('.table__body tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        data.append([col.text.strip() for col in cols])
    
    # Создание DataFrame
    df_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    df = pd.DataFrame(data, columns=df_columns)
    
    return df


data = get_race_data(2023, "Barcelona", "Main Race")

data.head()


,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,1,51,Pro Cup,"Alessio Rovera, Robert Shwartzman, Nicklas Nie...",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.540,88,
1,2,71,Pro Cup,"Antonio Fuoco, Daniel Serra, Davide Rigon",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.665,88,0.422
2,3,96,Pro Cup,"Thomas Preining, Laurin Heinrich, Dennis Olsen",Rutronik Racing,Porsche 911 GT3 R (992),1:41.562,88,2.328
3,4,777,Pro Cup,"Maro Engel, Luca Stolz, Fabian Schiller",Mercedes-AMG Team AlManar,Mercedes-AMG GT3 EVO,1:41.467,88,2.770
4,5,88,Pro Cup,"Raffaele Marciello, Jules Gounon, Timur Bogusl...",AKKODIS ASP Team,Mercedes-AMG GT3 EVO,1:42.162,88,4.413
